In [1]:
import altair as alt
import geopandas as gpd
import pandas as pd
import numpy as np
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

import os
os.chdir("..")

import sys
sys.path.insert(0, "src")

from viz import *

In [2]:
if not os.path.exists("viz"):
    os.mkdir("viz")

In [3]:
def sizeCode(x):
    if 1000 <= x < 5000:
        return "F"
    elif 5000 <= x < 10000:
        return "G"
    elif 10000 <= x < 50000:
        return "H"
    elif 50000 <= x < 100000:
        return "I"
    elif 100000 <= x:
        return "J+"
    else:
        return "<F"

    
def oobDate(x):
    """
    Returns None if Datetime object is out of bounds
    """
    try:
        x = pd.to_datetime(x)
        return x
    except:
        return None
    
    
def avgFiresBin(x):
    if x < 2:
        return "<2"
    elif 2 <= x < 5:
        return "<5"
    elif 5 <= x < 10:
        return "<10"
    elif x >= 10:
        return ">=10"

In [5]:
allFires = gpd.read_file("data/California_Fire_Perimeters_(all).geojson").to_crs("EPSG:3310")
allFires = allFires[~allFires["ALARM_DATE"].apply(oobDate).isna()]

allFires["month"] = pd.DatetimeIndex(allFires["ALARM_DATE"]).month
allFires["sizeCode"] = allFires["GIS_ACRES"].apply(sizeCode)

bigFires = allFires[allFires["GIS_ACRES"] >= 1000]

print(allFires.shape,
      bigFires.shape)

(14444, 21) (2638, 21)


In [ ]:
temp_df = pd.DataFrame(bigFires["sizeCode"].value_counts()).reset_index()

chart_1 = alt.Chart(temp_df
            ).mark_bar(
            ).encode(x=alt.X("index:O",
                             title="Fire Size Code"),
                     y=alt.Y("sizeCode:Q",
                             title="Number of Fires (1950-2020)"),
                     color="index:N",
                     tooltip=[alt.Tooltip("sizeCode",
                                          title="Number of Fires")]
            ).configure_axis(labelFontSize=12,
                             titleFontSize=15
            ).properties(width=500, height=400)

chart_1.save("viz/chart_1.png", scale_factor=2.0)

In [7]:
pivot_1 = allFires.pivot_table(index="YEAR_",
                               values="GIS_ACRES",
                               aggfunc=[len, sum, np.mean]
                 ).astype(int
                 ).reset_index()

pivot_1.columns = pivot_1.columns.droplevel()
pivot_1.columns = ["Year", "Number of Fires", "Burned Acres", "Average Size"]


base = alt.Chart(pivot_1
         ).encode(x=alt.X("Year:O", #year(Year):T
                          axis=alt.Axis(title=None,
                                        values=list(range(1950,2021,5))))
         ).properties(width=700, height=350)


alt.vconcat(base.mark_line(stroke="#92B4FF",
                           interpolate="monotone",
                           point=alt.OverlayMarkDef(color="blue", size=35)
               ).encode(y="Number of Fires:Q",
                        tooltip=["Year", "Number of Fires"]),
            base.mark_line(stroke="pink",
                           interpolate="monotone",
                           point=alt.OverlayMarkDef(color="red", size=35)
               ).encode(y=alt.Y("Burned Acres:Q"),
                        tooltip=["Year", "Burned Acres"]),
            base.mark_line(stroke="lightgreen",
                           interpolate="monotone",
                           point=alt.OverlayMarkDef(color="green", size=35)
               ).encode(y="Average Size:Q",
                        tooltip=["Year", "Average Size"])
  ).configure_axis(labelFontSize=13,
                   titleFontSize=15)

alt.VConcatChart(...)

In [12]:
chart_2 = base.mark_line(stroke='#92B4FF',
                         interpolate='monotone',
                         point=alt.OverlayMarkDef(color="blue", size=35)
             ).encode(y=alt.Y("Number of Fires:Q", title=" "),
                      tooltip=["Year", "Number of Fires"]
             ).properties(title="Number of Fires Per Year in California"
             ).configure_axis(labelFontSize=13,
                              titleFontSize=15)

chart_2.save("viz/chart_2.png", scale_factor=2.0)

In [ ]:
# heatmap of average fire size (>1000 acres)

pivot_3 = bigFires[bigFires["YEAR_"].astype(int) >=1985].pivot_table(index="YEAR_",
                            columns="month",
                            values="GIS_ACRES",
                            aggfunc=np.mean
              ).reset_index().melt(id_vars="YEAR_")

pivot_3["value"] = pivot_3["value"]//1000
pivot_3 = pivot_3[pivot_3["month"] >= 7]


heatmap = alt.Chart(pivot_3
            ).mark_rect(stroke="gray",
                        strokeWidth=1
            ).encode(x=alt.X("YEAR_:O",
                             title="Year"),
                     y=alt.Y("month:O",
                             title="Month"),
                     color=alt.Color("value:Q",
                                     legend=alt.Legend(title="Average Size"),
                                     scale=alt.Scale(scheme="purpleblue")),
                     tooltip=[alt.Tooltip("month", title="Month"),
                              alt.Tooltip("YEAR_", title="Year"),
                              alt.Tooltip("value", title="Average Size")])

text = heatmap.mark_text(baseline="middle", size=14
             ).encode(text="value:Q",
                      color=alt.condition((alt.datum.value == None) | (alt.datum.value > 50),
                                           alt.value("white"),
                                           alt.value("grey")))


chart_3 = alt.layer(heatmap, text
            ).properties(title="Average Fire Size",
                         width=830,
                         height=400
            ).configure_axis(labelFontSize=12,
                             titleFontSize=15)

chart_3.save("viz/chart_3.png", scale_factor=2.0)

In [ ]:
counties = gpd.read_file("data/CA_Counties/CA_Counties_TIGER2016.shp").to_crs("EPSG:3310")
sfLowerBound = counties[counties["NAME"]=="San Francisco"]["geometry"].bounds["maxy"].values[0]

norCalFires = allFires[allFires.bounds.apply(lambda x: x[1]>sfLowerBound, axis=1)]
norCalFires = norCalFires[norCalFires["YEAR_"].astype(int) >= 1980]

norCalCounties = counties[counties.bounds.apply(lambda x: x[3]>sfLowerBound, axis=1)]

norCalFires["centroid"] = norCalFires.centroid
countyGeo = {county: geometry for county, geometry in norCalCounties[["NAME", "geometry"]].values}

In [ ]:
fireCount = defaultdict(int)
burnedAcres = defaultdict(int)

for size, centroid in norCalFires[["GIS_ACRES", "centroid"]].values:
    for county, geometry in countyGeo.items():
        if centroid.within(geometry):
            fireCount[county] += 1
            burnedAcres[county] += size
            break
        else:
            continue
            
        
for county, count in fireCount.items():
    burnedAcres[county] = int(burnedAcres[county])

In [ ]:
norCalCounties["Number of Fires"] = norCalCounties["NAME"].apply(lambda x: fireCount[x])
norCalCounties["Burned Acres"] = norCalCounties["NAME"].apply(lambda x: burnedAcres[x])
norCalCounties["firesBin"]= norCalCounties["Number of Fires"].apply(lambda x: avgFiresBin(x/40))

norCalCounties = norCalCounties.to_crs("EPSG:4326")
norCalCounties["x"] = norCalCounties.centroid.x
norCalCounties["y"] = norCalCounties.centroid.y

In [ ]:
chart_4 = alt.layer(alt.Chart(norCalCounties
                      ).mark_geoshape(stroke="black"
                      ).encode(color=alt.Color("firesBin:O",
                                               legend=alt.Legend(title=None,
                                                                 orient='top-right',
                                                                 offset=0,
                                                                 labelFontSize=15),
                                               scale=alt.Scale(domain=["<2", "<5", "<10", ">=10"],
                                                               range=["rgb(254,224,156)", "rgb(254,186,109)",
                                                                      "rgb(255,146,97)", "rgb(255,102,88)"])),
                               tooltip=["NAME", "Number of Fires", "Burned Acres"]
                      ).properties(title="Average Fires per Year in Northern California (1980-2020)",
                                   width=800,
                                   height=800),
                    alt.Chart(norCalCounties
                      ).mark_text(
                      ).encode(longitude="x:Q",
                               latitude="y:Q",
                               text="NAME:N",
                               size=alt.value(11),
                               opacity=alt.value(1)))

chart_4.configure(style=dict(cell=dict(strokeOpacity=0))
      ).save("viz/chart_4.png", scale_factor=2.0)

In [ ]:
chart_5 = alt.layer(alt.Chart(norCalCounties
                      ).mark_geoshape(stroke="black"
                      ).encode(color=alt.Color("Burned Acres:Q",
                                               legend=alt.Legend(direction="vertical",
                                                                 orient='right',
                                                                 labelFontSize=12),
                                               scale=alt.Scale(scheme="orangered")),
                               tooltip=["NAME", "Number of Fires", "Burned Acres"]
                      ).properties(title="Burned Acres in Northern California (1980-2020)",
                                   width=800,
                                   height=800),
                    alt.Chart(norCalCounties
                      ).mark_text(
                      ).encode(longitude="x:Q",
                               latitude="y:Q",
                               text="NAME:N",
                               size=alt.value(11),
                               opacity=alt.value(1)))

chart_5.configure(style=dict(cell=dict(strokeOpacity=0))
      ).save("viz/chart_5.png", scale_factor=2.0)